#                PROJET MACHINE LEARNING LOQUET ANTOINE M2 OPHO 

Importation des bibliothèques nécéssaires : 
Importing the necessary libraries :

In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, validation_curve 
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

Importation du fichier : ( Mettre chemin d'accès du fichier roboBohr.csv) 
Import file: ( Set path to roboBohr.csv file )

In [ ]:
chemin_fichier = 'Mettre chemin d accès'
df = pd.read_csv(chemin_fichier)
df

La commande df permet d'avoir un rapide apercu du DataFrame, il sera mieux analysé par la suite
The df command gives you a quick overview of the DataFrame, which you can then analyze further.

# 1 Analyse du DataFrame

## 1.1 Analyse de forme

Analyse de la taille du DataFrame :
DataFrame size analysis :

In [ ]:
df.shape

La taille du DataFrame est de 16 242 lignes et 1 278 colonnes
DataFrame size is 16,242 rows and 1,278 columns

Analyse du nombre des différents types de colonnes : 
Analysis of the number of different column types : 

In [ ]:
df.dtypes.value_counts()

Parmi les 1 278 colonnes, 2 sont de type int64 et le reste des float64. On voit ici que nous sommes dans un problème de type régression et de de classification
Of the 1,278 columns, 2 are int64 and the rest float64. We can see here that we're dealing with a regression and classification problem.

Analyse plus précise des colonnes de type int64
More precise analysis of int64 columns

In [ ]:
colonnes_int64 = df.select_dtypes(include='int64')
print(colonnes_int64)

Les int64 ont pour nom Unnamed: 0 et pubchem_id, elles ne semblent pas intéréssantes pour la suite
The int64s are Unnamed: 0 and pubchem_id, and don't seem to be of interest for the rest of the project.

Vérification de la présence de NaN : 
Checking for the presence of NaN : 

In [ ]:
nombre_de_nans = df.isna().sum().sum()
print(nombre_de_nans)

Il n'y a pas de NaN dans le DataFrame
There is no NaN in the DataFrame

Calcul du nombre de cases nulles dans le DataFrame : 
Calculating the number of null cells in the DataFrame : 

In [ ]:
nombre_de_zero = (df==0.0 ).sum().sum()
print(nombre_de_zero/(df.shape[0]*df.shape[1])*100)

71 % des valeurs sont des 0 
71% of values are 0 

## 1.2 Analyse de fond 

In [ ]:
X=df.drop(['pubchem_id','Unnamed: 0', 'Eat'], axis=1)
Y=df['Eat']

Séparation du DataFrame en deux, X pour les variables et Y pour les outputs. Les colonnes inutiles sont également retirées
Separation of the DataFrame into X for variables and Y for outputs. Unnecessary columns are also removed

In [ ]:
Y.describe()

Données statistiques de Y qui nous serviront pour la comparation finale des modèles 
Y statistical data for final model comparison 

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(df['Eat'],kde=True, color='blue')
plt.title('Histogramme des valeurs')
plt.xlabel('Valeurs')
plt.ylabel('Fréquence')
plt.show()

Histogramme de Y, qui nous serivra aussi pour comparer les resultats des modèles 
Histogram of Y, which we can also use to compare model results. 

Standardisation de X :
Standardizing X :

In [ ]:
X_stand = pd.DataFrame(StandardScaler().fit_transform(X)) #moyenne nulle et un écart type de 1

Normalisation de X : 
Normalization of X : 

In [ ]:
X_stand_norm_l1=pd.DataFrame(normalize(X_stand, norm='l1', axis=0)) 
X_stand_norm_l2=pd.DataFrame(normalize(X_stand, norm='l2', axis=0)) 
X_stand_norm_max=pd.DataFrame(normalize(X_stand, norm='max', axis=0)) 
max_line_l1 = X_stand_norm_l1.describe().loc['mean']
max_line_l2 = X_stand_norm_l2.describe().loc['mean']
max_line_max = X_stand_norm_max.describe().loc['mean']

# Créer une nouvelle figure avec trois sous-graphiques
fig, axs = plt.subplots(3, 1, figsize=(10, 15))

# Tracer la ligne maximale pour la norme L1
axs[0].plot(max_line_l1.index, max_line_l1.values, marker='o', linestyle='-', color='red')
axs[0].set_xlabel('Colonnes')
axs[0].set_ylabel('Valeurs maximales')
axs[0].set_title('Norme L1')

# Tracer la ligne maximale pour la norme L2
axs[1].plot(max_line_l2.index, max_line_l2.values, marker='o', linestyle='-', color='blue')
axs[1].set_xlabel('Colonnes')
axs[1].set_ylabel('Valeurs maximales')
axs[1].set_title('Norme L2')

# Tracer la ligne maximale pour la norme max
axs[2].plot(max_line_max.index, max_line_max.values, marker='o', linestyle='-', color='green')
axs[2].set_xlabel('Colonnes')
axs[2].set_ylabel('Valeurs maximales')
axs[2].set_title('Norme max')

# Ajuster la disposition des sous-graphiques
plt.tight_layout()

# Afficher la figure
plt.show()

J'ai constaté qu'il existait plusieurs normalisations, j'ai alors décidé de les comparer graphiquement. Je constate que la normalisation L1 donne une moyenne moins fluctuante, c'est pourquoi je l'ai choisie.

I found that there were several normalizations, so I decided to compare them graphically. I found that the L1 normalization gave a less fluctuating average, which is why I chose it.

In [ ]:
pca = PCA(n_components=0.99)
X_pca = pd.DataFrame(pca.fit_transform(X_stand_norm_l1)) 

PCA Permet de reduire la taille de X à 15 colonnes (valeur déterminée ci dessous) au lieu de 1275 en gardant 99% des informations

PCA Reduces the size of X to 15 columns (value determined below) instead of 1275, keeping 99% of the information.

In [ ]:
new_df=X_pca.join(Y)

In [ ]:
train_set, test_set = train_test_split(new_df, test_size=0.2, random_state=0) 
#Séparation des données pour entrainer une partie et pour tester le modèle 

Séparation du DataFrame réduit en deux set, un train set et un test set pour nos modèles. La répartition des données est 20% pour le train set et 80% pour le test set. J'ai placé un random_state pour pouvoir comparer mes resultats malgré plusieurs relance

Separation of the reduced DataFrame into two sets, a train set and a test set for our models. Data distribution is 20% for the train set and 80% for the test set. I've set a random_state so that I can compare my results despite several relaunches.

In [ ]:
X_train=train_set.drop(['Eat'], axis=1)
Y_train=train_set['Eat']
X_test=test_set.drop(['Eat'], axis=1)
Y_test=test_set['Eat']

Séparation des variables et des outputs pour les deux sets 
Separation of variables and outputs for both sets 

Analyse des colonnes du train set : 
Analysis of train set columns : 

In [ ]:
train_set.hist(figsize=(15,15))

L'histogramme est similaire à Y et les valeurs des colonnes sont centrées autour de 0, ce qui montre que la standisation et la normalisation sont conservées. 

The histogram is similar to Y, and the column values are centered around 0, showing that standardization and normalization are preserved. 

# 2 Machine Learning 

## 2.1 Fonctions

Création d'une fonction qui va permettre d'évaluer les modèles avant et sans optmisation des hyperparamètres : 
Creation of a function to evaluate models before and without optmization of hyperparameters: 

In [ ]:
def evaluation(model_class, X_train, Y_train, X_test, Y_test, best_params=None):

    # Vérifier si des paramètres optimaux sont fournis
    if best_params is None:
        # Créer une instance du modèle avec les paramètres par défaut
        model = model_class()
        # Entraîner le modèle sur les données d'entraînement
        model_train = model.fit(X_train, Y_train)
        # Calculer le score d'entraînement
        train_score = model.score(X_train, Y_train)
        # Calculer le score de test
        test_score = model.score(X_test, Y_test)
        
        # Afficher les scores et l'écart entre le score d'entraînement et de test
        print("Score d'entraînement : ", train_score, '\n'
              "Score de test : ", test_score, '\n'
              "Écart : ", train_score - test_score)

        # Vérifier s'il y a un surajustement (overfitting)
        if train_score < test_score:
            print("Surajustement, paramètres à ajuster")
        
        # Renvoyer le modèle entraîné et les scores
        return model_train, train_score, test_score
    else:
        # Créer une instance du modèle avec les meilleurs paramètres fournis
        best_model = model_class(**best_params)
        # Entraîner le modèle sur les données d'entraînement
        best_model_train = best_model.fit(X_train, Y_train)
        # Calculer le score d'entraînement
        best_train_score = best_model.score(X_train, Y_train)
        # Calculer le score de test
        best_test_score = best_model.score(X_test, Y_test)
        
        # Afficher les scores et l'écart entre le score d'entraînement et de test
        print("Score d'entraînement : ", best_train_score, '\n'
              "Score de test : ", best_test_score, '\n'
              "Écart : ", best_train_score - best_test_score)
        
        # Vérifier s'il y a un surajustement (overfitting)
        if best_train_score < best_test_score:
            print("Surajustement, paramètres à ajuster")
            
        # Renvoyer le modèle entraîné avec les meilleurs paramètres et les scores
        return best_model_train, best_train_score, best_test_score


 Création d'une fonction qui trouve les meilleurs hyperparamètres pour un modèle en utilisant la recherche sur grille : 
 Create a function that finds the best hyperparameters for a model using grid search : 

In [ ]:
def best_param(model, X_train, Y_train, param_grid):

    # Créer une instance de GridSearchCV avec le modèle, la grille d'hyperparamètres, la validation croisée et la métrique de performance
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
    # Effectuer la recherche sur grille sur les données d'entraînement
    grid_search.fit(X_train, Y_train)
    
    # Obtenir les meilleurs hyperparamètres trouvés par la recherche sur grille
    best_params = grid_search.best_params_
    # Afficher les meilleurs hyperparamètres
    print("Meilleurs hyperparamètres :", best_params)
    
    # Renvoyer les meilleurs hyperparamètres
    return best_params


Création d'une fonction qui vérifie si l'ajustement a fonctionné ou non :
Create a function that checks whether the adjustment has worked or not:

In [ ]:
def Verification_opti(best_train, best_test, train, test):

    # Vérifier si le score sur l'ensemble d'entraînement est optimisé
    if best_train > train:
        print('Le set train est optimisé')
    else:
        print('Le set train n\'est pas optimisé')

    # Vérifier si le score sur l'ensemble de test est optimisé
    if best_test > test:
        print('Le set test est optimisé')
    else:
        print('Le set test n\'est pas optimisé')


Création d'une fonction qui vérifie la robustesse du modèle avec la validation croisée : 
Creation of a function that checks the robustness of the model with cross-validation: 

In [ ]:
def robustesse(modele):
    num_folds = 7

    # Créer un objet KFold pour spécifier la stratégie de validation croisée
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Utiliser cross_val_score pour obtenir les scores de chaque fold avec R2 score comme métrique
    r2_scorer = make_scorer(r2_score)  # Créer un objet scorer pour R2 score
    scores_r2 = cross_val_score(modele, X_train, Y_train, scoring=r2_scorer, cv=kf)

    # Afficher les scores R2 pour chaque fold
    print('Scores R2 pour chaque fold :', scores_r2)

    # Calculer la moyenne des scores R2
    average_r2 = np.mean(scores_r2)

    print(f'Moyenne R2 sur {num_folds} folds : {average_r2}')

## 2.2 Modèles 

### 2.2.1 LinearRegression

Evaluation du modèle LinearRegression : 
Evaluation of the LinearRegression model : 

In [ ]:
Reg_model, Reg_train_score, Reg_test_score = evaluation(LinearRegression, X_train, Y_train, X_test, Y_test)

Pas d'overfitting, et un R2 score un peu juste. Vérifions avec la validation croisée, si le score reste identique ou non pour déterminer la robustesse du modèle (Je chercher seulement la robustesse de la meilleure version du modèle) : 

No overfitting, and an R2 score that's just right. Let's check with cross-validation whether the score remains the same or not to determine the robustness of the model (I'm only looking for the robustness of the best version of the model): 

In [ ]:
robustesse(Reg_model)

Le modèle est robuste dans son ensemble. Il peut être nécessaire d'explorer d'autres approches de modélisation, d'ajuster les hyperparamètres, ou de considérer des caractéristiques supplémentaires pour améliorer davantage les performances du modèle. La grille ci dessous, va essayer d'améliorer le score : 

The model is robust overall. It may be necessary to explore other modeling approaches, adjust hyperparameters, or consider additional features to further improve model performance. The grid below will attempt to improve the score: 

In [ ]:
param_grid_reg = {
    'fit_intercept': [True, False],
    'positive': [True, False]
}

La grille étant définit, il faut chercher la meilleure combinaison qui donnera l'erreur quadratique moyenne négative la plus faible : 
Now that the grid has been defined, we need to look for the best combination that will give the lowest negative root mean square error: 

In [ ]:
best_param_reg = best_param(Reg_model, X_train, Y_train, param_grid_reg)

Les meilleurs hyperparamètres sont : {'fit_intercept': True, 'positive': False}. Nouvelle évaluation avec ces nouveaux paramètres :

The best hyperparameters are: {'fit_intercept': True, 'positive': False}. New evaluation with these new parameters :

In [ ]:
best_reg_model, best_reg_train_score, best_reg_test_score = evaluation(LinearRegression, X_train, Y_train, X_test, Y_test, best_param_reg)

Pas d'overfitting, et un R2 score toujours un peu juste. Vérification des l'optimisation :

No overfitting, and an R2 score that's still a little tight. Optimization check :

In [ ]:
Verification_opti(best_reg_train_score,best_reg_test_score, Reg_train_score, Reg_test_score)

Aucune plus value de GridSearch pour ce modèle 
No added value from GridSearch for this model 

### 2.2.2 RandomForest

J'ai procédé de la sorte pour tout mes modèles, je vais désormais seulement commenter les résultats 
I've done this for all my models, and from now on I'll just comment on the results. 

In [ ]:
Forest_model, Forest_train_score, Forest_test_score =evaluation(RandomForestRegressor, X_train, Y_train, X_test, Y_test)

R2 score très bon, et pas d'overtfitting. Les temps de calculs sont longs pour mon ordinateur pour l'optimisation de la grille de paramètres. J'ai décidé de couper les paramètres en deux sets. J'ai calculé le set 1 seul, puis le set 2 avec les paramètres du set 1 calculés. Ce n'est peut-être pas la méthode optimale et les résultats changeraient peut-être

R2 score very good, and no overtfitting. My computer is taking a long time to optimize the parameter grid. I decided to split the parameters into two sets. I calculated set 1 alone, then set 2 with the calculated set 1 parameters. This may not be the optimum method, and the results may change.

In [ ]:
param_grid_Forest = {    
    
    #Set 1  
    'n_estimators': [50, 100, 200],  # Nombre d'arbres dans la forêt
    'max_features': ['auto', 'sqrt', 'log2'],  # Nombre maximum de fonctionnalités considérées pour la scission
    'max_depth': [None, 10, 20, 30],  # Profondeur maximale de chaque arbre 
    
    #Set 2 
    'min_samples_split': [2, 5, 10],  # Nombre minimum d'échantillons requis pour scinder un nœud interne
    'min_samples_leaf': [1, 2, 4],  # Nombre minimum d'échantillons requis pour être une feuille
    'bootstrap': [True, False]  # Méthode d'échantillonnage pour la construction d'arbres   

}

In [ ]:
best_param_Forest = best_param(Forest_model, X_train, Y_train, param_grid_Forest)

Les meilleurs hyperparamètres sont : {'bootstrap': False, 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

The best hyperparameters are : {'bootstrap': False, 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [ ]:
best_Forest_model, best_Forest_train_score, best_Forest_test_score = evaluation(RandomForestRegressor, X_train, Y_train, X_test, Y_test, best_param_Forest)

R2 score très bon une nouvelle fois et pas d'overfitting

R2 very good score once again and no overfitting

In [ ]:
Verification_opti(best_Forest_train_score,best_Forest_test_score,Forest_train_score, Forest_test_score)

RandomForest est un bon model, et bien optmisé par GridSearch. Cependant curieusement seul le train set est optmisé

RandomForest is a good model, and well opted for by GridSearch. Curiously, however, only the train set is optmized.

In [ ]:
robustesse(best_Forest_model)

Le score reste le même sensiblement le même, le modèle est robuste.
The score remains more or less the same, and the model is robust.

### 2.2.3 SVR

In [ ]:
SVR_model, SVR_train_score, SVR_test_score =evaluation(SVR, X_train, Y_train, X_test, Y_test)

R2 score correct, mais il y a de l'overfitting. La grille pourra peut-être fixer ce problème
R2 correct score, but there's some overfitting. The grid may be able to fix this problem

Une nouvelle fois les temps de calculs sont longs. J'ai procédé de la même manière que 2.2.2 
Once again, calculation times are long. I proceeded in the same way as 2.2.2 

In [ ]:
param_grid_SVR = {
    
    #Set 1  
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Type de noyau à utiliser dans le modèle SVM
    'C': [0.1, 1, 10],  # Paramètre de régularisation, contrôle la pénalité pour les erreurs d'entraînement
    
    #Set 2 
    'gamma': ['scale', 'auto', 0.1, 1],  # Paramètre du noyau (pour les noyaux RBF, poly et sigmoïde)
    'epsilon': [0.1, 0.2, 0.5]  # Détermine la largeur de la marge autour de la valeur de régression attendue

}

In [ ]:
best_param_SVR = best_param(SVR_model, X_train, Y_train, param_grid_SVR)

Les meilleurs hyperparamètres sont : {'C': 10, 'epsilon': 0.2, 'gamma': 'scale', 'kernel': 'rbf'}
The best hyperparameters are: {'C': 10, 'epsilon': 0.2, 'gamma': 'scale', 'kernel': 'rbf'}.

In [ ]:
best_SVR_model, best_SVR_train_score, best_SVR_test_score = evaluation(SVR, X_train, Y_train, X_test, Y_test, best_param_SVR)

In [ ]:
Verification_opti(best_SVR_train_score,best_SVR_test_score,SVR_train_score, SVR_test_score)

Score amélioré, overfitting réduit mais toujours présent.
Score improved, overfitting reduced but still present.

In [ ]:
robustesse(best_SVR_model)

Le score reste le même sensiblement le même, le modèle est robuste.
The score remains more or less the same, and the model is robust.

### 2.2.4 AdaBoostRegressor

In [ ]:
ABR_model, ABR_train_score, ABR_test_score =evaluation(AdaBoostRegressor, X_train, Y_train, X_test, Y_test)

R2 score correct, et pas d'overfitting
R2 correct score, no overfitting

In [ ]:
param_grid_ABR = {
   'n_estimators': [50, 100, 200],  # Nombre d'estimateurs (régresseurs faibles) dans la chaîne
   'learning_rate': [0.01, 0.1, 0.5, 1.0],  # Taux d'apprentissage, contrôle la contribution de chaque estimateur
   'loss': ['linear', 'square', 'exponential'],  # Fonction de perte à optimiser lors de la mise à jour des poids
}


In [ ]:
best_param_ABR = best_param(ABR_model, X_train, Y_train, param_grid_ABR)

Les meilleurs hyperparamètres sont : {'learning_rate': 0.1, 'loss': 'exponential', 'n_estimators': 200}
The best hyperparameters are: {'learning_rate': 0.1, 'loss': 'exponential', 'n_estimators': 200}

In [ ]:
best_ABR_model, best_ABR_train_score, best_ABR_test_score = evaluation(AdaBoostRegressor, X_train, Y_train, X_test, Y_test, best_param_ABR)

In [ ]:
Verification_opti(best_ABR_train_score,best_ABR_test_score,ABR_train_score, ABR_test_score)

Modèle correct dans son score, et amélioration legère par GridSearchCV
Correct model score, slightly improved by GridSearchCV

In [ ]:
robustesse(best_ABR_model)

Le score varie peu, le modèle est robuste
The score varies little, the model is robust

### 2.2.5 DecisionTreeRegressor

In [ ]:
DTR_model, DTR_train_score, DTR_test_score =evaluation(DecisionTreeRegressor, X_train, Y_train, X_test, Y_test)

Très bon R2 score et pas d'overfitting
Very good R2 score and no overfitting

In [ ]:
param_grid_DTR = {    
   'max_depth': [None, 5, 10, 20],  # Profondeur maximale de l'arbre
   'min_samples_split': [2, 5, 10],  # Nombre minimum d'échantillons requis pour scinder un nœud interne
   'min_samples_leaf': [1, 2, 4],  # Nombre minimum d'échantillons requis pour être une feuille
   'max_features': ['sqrt', 'log2'],  # Nombre maximum de fonctionnalités considérées pour la scission
}

In [ ]:
best_param_DTR = best_param(DTR_model, X_train, Y_train, param_grid_DTR)

Les meilleurs hyperparamètres sont : {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10}
The best hyperparameters are: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10}

In [ ]:
best_DTR_model, best_DTR_train_score, best_DTR_test_score = evaluation(DecisionTreeRegressor, X_train, Y_train, X_test, Y_test, best_param_DTR)

In [ ]:
Verification_opti(best_DTR_train_score,best_DTR_test_score,DTR_train_score, DTR_test_score)

Le modèle est très bon et fonctionne mieux avec les paramètres par défauts
The model is very good and works best with the default settings.

In [ ]:
robustesse(best_DTR_model)

Le score ne change pas de manière significatif, le modèle est robuste
Score does not change significantly, model is robust

### 2.2.6 KNeighborsRegressor

In [ ]:
KNR_model, KNR_train_score, KNR_test_score =evaluation(KNeighborsRegressor, X_train, Y_train, X_test, Y_test)

Très bon R2 score et pas d'overfitting
Very good R2 score and no overfitting

In [ ]:
param_grid_KNR = {
    'n_neighbors': [3, 5, 7, 9],  # Nombre de voisins à considérer
    'weights': ['uniform', 'distance'],  # Poids donnés aux voisins (uniformes ou inverses de la distance)
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithme utilisé pour calculer les voisins
    'leaf_size': [10, 20, 30],  # Taille de la feuille pour les arbres (utilisé avec ball_tree ou kd_tree)
}



In [ ]:
best_param_KNR = best_param(KNR_model, X_train, Y_train, param_grid_KNR)

Les meilleurs hyperparamètres sont : {'algorithm': 'auto', 'leaf_size': 10, 'n_neighbors': 3, 'weights': 'distance'}
The best hyperparameters are: {'algorithm': 'auto', 'leaf_size': 10, 'n_neighbors': 3, 'weights': 'distance'}.

In [ ]:
best_KNR_model, best_KNR_train_score, best_KNR_test_score = evaluation(KNeighborsRegressor, X_train, Y_train, X_test, Y_test, best_param_KNR)

In [ ]:
Verification_opti(best_KNR_train_score,best_KNR_test_score,KNR_train_score, KNR_test_score)

Bon modèle et fonctionne mieux quand il est optimisé
Good model and works best when optimized

In [ ]:
robustesse(best_KNR_model)

Le model est robuste car les scores ne changent peu.
The model is robust because the scores change very little.

## 2.3 Résultats 

Prédiction des données à l'aide des différents modèles
Data prediction using different models

In [ ]:
Y_pred_Reg = Reg_model.predict(X_test)
Y_pred_Forest = best_Forest_model.predict(X_test)
Y_pred_SVR = best_SVR_model.predict(X_test)
Y_pred_ABR = best_ABR_model.predict(X_test)
Y_pred_DTR = DTR_model.predict(X_test)
Y_pred_KNR = best_KNR_model.predict(X_test)

Comparaison des histogrammes prédits par les modèles :

Comparison of histograms predicted by the models :

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 8))
fig.suptitle('Comparaison des prédictions des modèles')

# Liste des prédictions et des labels correspondants
predictions = [Y_pred_Reg, Y_pred_Forest, Y_pred_SVR, Y_pred_ABR, Y_pred_DTR, Y_pred_KNR]
labels = ['Linear', 'RandomForest', 'SVR', 'AdaBoost', 'DecisionTree', 'KNeighbors']

# Afficher les histogrammes normalisés des prédictions dans des sous-graphiques
for pred, label, ax in zip(predictions, labels, axes.flatten()):
    ax.hist(Y, bins=30, alpha=0.5, label='Y', density=True)
    ax.hist(pred, bins=30, alpha=0.5, label=f'Y_pred_{label}', density=True)    
    ax.set_title(f'Modèle {label}')
    ax.set_xlabel('Valeurs')
    ax.set_ylabel('Densité de probabilité')
    ax.legend()

# Ajuster la disposition des sous-graphiques
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Afficher la figure
plt.show()

On voit nettement que les modèles AdaBoost et Linear ne reproduisent pas fidèlement l'histogramme initial. Le modèle SVR ne repoduit pas très bien le centre et la queue droite. Concerant les trois dernièrs, bien qu'il y ait une surévaluation par endroits, les modèles semblent approprier pour prédire les données. Pour cela, je vais déterminer avec un score par la suite le meilleur

We can clearly see that the AdaBoost and Linear models do not faithfully reproduce the original histogram.The SVR model does not reproduce the center and right tail very well. With regard to the last three, although there is an overestimation in places, the models seem to be suitable for predicting the data. To this end, I'm going to determine with a score the best

Affichage des statistiques des prédites à celles des données originale: 

Display statistics from predicted to original data:

In [ ]:
model_predictions = {
    'Y_pred_Reg': Reg_model.predict(X_test),
    'Y_pred_Forest': best_Forest_model.predict(X_test),
    'Y_pred_SVR': best_SVR_model.predict(X_test),
    'Y_pred_ABR': best_ABR_model.predict(X_test),
    'Y_pred_DTR': DTR_model.predict(X_test),
    'Y_pred_KNR': best_KNR_model.predict(X_test)
}

# Créer le DataFrame avec les prédictions
predictions_df = pd.DataFrame(model_predictions)

# Obtenir les statistiques descriptives des prédictions
predictions_stats = predictions_df.describe()

# Créer le DataFrame de comparaison
comparison_stats = pd.concat([df['Eat'].describe(), predictions_stats], axis=1)
comparison_stats.columns = ['Y'] + list(model_predictions.keys())

# Afficher le DataFrame de comparaison
comparison_stats

Dans l'ensemble la moyenne, la déviation standard, le premier quartile, le deuxième quartile et le troisième quartile semblent correctement prédits. Le minimum et le maximum ne sont cependant pas correctement prédits par le modèle Linear Regression et et AdaBoost Regression, ce qui etait visible sur les histogramme

Overall, the mean, standard deviation, first quartile, second quartile and third quartile appear to be correctly predicted. The minimum and maximum, however, are not correctly predicted by the Linear Regression and AdaBoost Regression models, as can be seen from the histograms.

Autre visualisation des données prédites, cette fois-ci, les erreurs absolues sont représentées :

Another visualization of predicted data, this time showing absolute errors:

In [ ]:
error_stats = pd.DataFrame(index=['mean', 'std', 'min', '25%', '50%', '75%', 'max'])

for model_name, model_prediction in model_predictions.items():
    # Calculer l'erreur absolue pour chaque statistique descriptive
    error_stats[f'Absolute_Error_{model_name}'] = abs(predictions_stats[model_name] - df['Eat'].describe())*100

# Afficher le DataFrame des erreurs absolues des statistiques descriptives
error_stats

On constate que les modèles prédisent plus ou moins bien. C'est pourquoi je vais leur attribuer un score. Ligne par ligne, le modèle ayant l'erreur la plus faible obtient un score de 1 et ainsi de suite jusqu'à l'erreur la plus grande. Au final, le modèle avec le plus faible score, sera le modèle qui a fait les erreurs les plus faibles devant les autres.

We can see that the models predict more or less well. That's why I'm going to give them a score. Line by line, the model with the lowest error gets a score of 1, and so on down to the highest error. In the end, the model with the lowest score will be the one with the smallest errors, ahead of the others.

In [ ]:
# Initialiser les scores à 0 dans un dictionnaire
model_scores = {model: 0 for model in model_predictions.keys()}

# Parcourir chaque ligne du DataFrame
for index, row in error_stats.iterrows():
    # Trier les colonnes par valeur
    sorted_columns = row.sort_values().index
    
    # Attribuer des scores de 1 à 6 en fonction de la position des colonnes
    assigned_positions = set()  # Garder une trace des positions déjà attribuées
    for position, model_column in enumerate(sorted_columns, start=1):
        # Extraire le nom du modèle à partir de la colonne
        model_name = model_column.replace('Absolute_Error_', '')
        
        # Gérer l'égalité en ajoutant la position suivante non attribuée
        while position in assigned_positions:
            position += 1
        
        # Ajouter le score à la position du modèle dans le dictionnaire
        model_scores[model_name] += position
        assigned_positions.add(position)


# Trier le dictionnaire par valeurs (scores) et afficher l'ordre croissant
sorted_scores = sorted(model_scores.items(), key=lambda x: x[1])
print("Scores attribués à chaque modèle :")
for model, score in sorted_scores:
    print(f"{model} : {score}")

Ainsi sont donnés dans l'ordre le meilleur modèle au pire : KNeighborsRegressor, RandomForestRegressor, DecisionTreeRegressor, SVR, Linear Regressor, AdaBoostRegressor. AdaBoostRegressor n'était pas le modèle avec le pire score, mais est celui qui reproduit le moins bien les données statistiques attendues. Cela montre que un autre score serait peut-être plus adapté dans notre situation.

In order of best to worst, the following models are given: KNeighborsRegressor, RandomForestRegressor, DecisionTreeRegressor, SVR, Linear Regressor, AdaBoostRegressor. AdaBoostRegressor was not the model with the worst score, but it is the one that reproduces the expected statistical data least well. This suggests that another score might be more appropriate in our situation.